<h1 style="color:blue;">Redes Neuronales Convolucionales para la detección de puntos anatómicos en imágenes cefálicas laterales</h1>

<p>En este proyecto busco utilizar algunas arquitecturas de redes neuronales convolucionales para determinar cual de ellas se adecúa mejor a la detección de puntos anatómicos en imágenes radiográficas cefálicas laterales, intentaré hacerlo lo mas sencillo posible.</p>

<h2 style="color:red;">Data Utilizada</h2>

<p>Para realizar esta investigación, voy a usar una data disponible de forma gratuita en <a href="http://www-o.ntust.edu.tw/~cweiwang/ISBI2015/challenge1/index.html">Dataset</a>, de la misma, solo utilice la data con labels "senior".</p>

<h3>Breve Descripción de la data</h3>

<p>Los datos a utilizar consisten en 400 imágenes de imágenes radiográficas cefálicas laterales (escala de grises) junto con un vector que indica la posición de 19 puntos anatómicos, etiquetados por un especialista senior en el área (ver link del dataset para mayor detalle).</p>

<h2 style="color:red;">Preprocesamiento y Data Augmentation</h2>

<p>Para hacer el modelo más general, las imágenes fueron preprocesadas para cumplir con:</p>

* Imágenes cuadradas de 128x128 pixeles.
* Imágenes normalizadas (rango de intensidad de pixeles de -1 a 1).
* Para tener mayor cantidad de datos y evitar overfit, se aumentó la data al hacer flip horizontal y vertical, para un total de 1200 imágenes con sus labels (400 originales + 400 flip horizontal + 400 flip vertical).
* Igualmente, para evitar overfit, las imágenes están organizadas en 1 original - 1 flip h - 1 flip v. Nota: Es cierto que lo ideal sería mezclar todo, sin embargo, las imágenes "flipeadas" distan tanto de la original que no creo que el efecto sea tan evidente, de todas formas es un punto a explorar.
* El training set consta de 900 imágenes, mientras que el test set de 300 imágenes.


<p>Tomemos una imagen de ejemplo junto con sus labels:</p>

<img src="imagen1.png" alt="Drawing" style="width: 450px;"/>

<p>Naturalmente, la imagen no es nítida debido a que tiene solo 128x128 píxeles de resolución. Los puntos rojos representan los puntos anatómicos etiquetados por el especialista.</p>

<h2 style="color:red;">Entorno de Ejecución</h2>

<p>Debido a diversas limitantes, la mejor opción que tengo disponible es, sin dudas, usando el entorno acelerado por GPU en Google Colab (no se imaginan cuanto lo agardezco), la forma de setearlo la obtuve de este post de <a href="https://www.kdnuggets.com/2018/02/google-colab-free-gpu-tutorial-tensorflow-keras-pytorch.html">KDnuggets.</a> Recomiendo ampliamente darle una hojeada para entender mejor como funciona.<br><br>
    Para hacer mas sencillo el código, se utilizará Keras con backend TensorFlow, lo cual hace el modelo muy fácil de armar, depurar y leer. La data está almacenada en mi Google Drive, la compartiré en cuanto esté seguro que tengo permiso de hacerlo, igual la data original es accesible en el link de la sección "Data Utilizada".</p>


<h1>Estableciendo la conexión con nuestro Google Drive</h1>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

<p>El código anterior tendrá dos puntos en los que nos indica un link al cual debemos acceder para iniciar nuestra sesión en Google y darle autorización a nuestra aplicación para acceder al Drive. Es tan sencillo como copiar los códigos de autenticación en los campos indicados.</p>

<p>Procedemos a montar nuestro Drive a través de la referencia "drive"</p>

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

<p>Ahora, verificamos que nuestro entorno en Google Colab tiene configurado el entorno acelerado por GPU, lo cual nos debería arrojar <b>['/device:GPU:0']</b> en caso afirmativo.</p>

<p><b>Nota: </b> El programa debe funcionar aun cuando no se encuentre acelerado por GPU, sin embargo, la diferencia en rapidez (y por tanto en viabilidad de la investigación) es muy considerable en mi caso, siendo alrededor de 20 a 30 veces más rápido, sin contar el hecho de que el computador no queda "esclavizado" por el uso exhaustivo de CPU y Memoria RAM.</p>

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

<h1>Importando librerías necesarias y cargando el training y test set</h1>

In [ ]:
!pip install -q keras #En caso de que no lo esté

import pickle
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, concatenate
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras.optimizers
from keras.utils import plot_model
import h5py

<p>Cargamos la data desde nuestro Drive en un diccionario con dos índices princiaples:</p>

* Diccionario['Img'] para obtener las imágenes (matriz de intensidad de grises).
* Diccionario['Lbl'] para obtener los labels.

<p>Es importante notar que índices iguales corresponden al par Imagen-Label respectivo, esto es, Diccionario['Img'][15] es una imagen cuyo label está guardado en Diccionario['Lbl'][15].</p>
    
<p>La ruta dentro de mi sesión de Google Drive para obtener la data a evaluar es "drive/ColabRuns/ImagenesFlipLabelsSenior128.pickle". Como se puede notar, es un archivo tipo 'pickle', para facilitar su lectura y por ser ligero en espacio que ocupa.</p>

In [ ]:
Diccionario = pickle.load(open("drive/ColabRuns/ImagenesFlipLabelsSenior128.pickle","rb"))

<p>Definimos nuestras variables de training y test según lo especificado anteriormente<p>

In [ ]:
X_train=np.array(Diccionario['Img'][0:900]).reshape(900,128,128,1).astype('float32')
Y_train=np.array(Diccionario['Lbl'][0:900]).reshape(900,38).astype('float32')

X_test=np.array(Diccionario['Img'][900:1200]).reshape(300,128,128,1).astype('float32')
Y_test=np.array(Diccionario['Lbl'][900:1200]).reshape(300,38).astype('float32')

<p>Algunas notas sobre el reshape y el astype:</p>
* reshape(900,128,128,1) corresponde a 900 imágenes, de 128 x 128 pixeles, de 1 canal cada una.
* reshape(900,38) corresponde a 900 labels, cada uno de 38 elementos (19 pares de coordenadas).
* astype('float32') es un tipo de dato aceptado por TensorFlow y Keras, cuando se guardó la data en un archivo 'pickle', los datos pueden no ser devueltos en un formato aceptable.

<p>Una vez cumplido lo explicado anteriormente, llegamos al punto en que debemos definir la arquitectura de red neuronal convolucional en si, en este caso, usaremos una arquitectura del tipo "AlexNet", que no es mas que una arquitectura secuencial, pero que se reconoce como el punto de inflexión que hizo posible el actual desarrollo y uso de las CNNs en el área de visión por computador.</p>

<p><b>Nota: </b>Las demás arquitecturas a evaluar serán exactamente iguales hasta aquí.</p>

<h1>Definición del Modelo: VGG19</h1>

<p>El modelo está basado en el trabajo de K. Simonyan y A. Zisserman:</p>
<br><b>
Very Deep Convolutional Networks for Large-Scale Image Recognition. K. Simonyan, A. Zisserman
arXiv:1409.1556
</b><br>

In [ ]:
input1 = Input(shape=(128,128,1))

Conv1 = Convolution2D(64, kernel_size=3, strides=1, activation='relu', padding='same')(input1)
Conv2 = Convolution2D(64, kernel_size=3, strides=1, activation='relu', padding='same')(Conv1)
Maxp1 = MaxPooling2D(pool_size=(2,2))(Conv2)

Conv3 = Convolution2D(128, kernel_size=3, strides=1, activation='relu', padding='same')(Maxp1)
Conv4 = Convolution2D(128, kernel_size=3, strides=1, activation='relu', padding='same')(Conv3)
Maxp2 = MaxPooling2D(pool_size=(2,2))(Conv4)

Conv5 = Convolution2D(256, kernel_size=3, strides=1, activation='relu', padding='same')(Maxp2)
Conv6 = Convolution2D(256, kernel_size=3, strides=1, activation='relu', padding='same')(Conv5)
Maxp3 = MaxPooling2D(pool_size=(2,2))(Conv6)

Conv7 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Maxp3)
Conv8 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Conv7)
Conv9 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Conv8)
Conv10 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Conv9)
Maxp4 = MaxPooling2D(pool_size=(2,2))(Conv10)

Conv11 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Maxp4)
Conv12 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Conv11)
Conv13= Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Conv12)
Conv14 = Convolution2D(512, kernel_size=3, strides=1, activation='relu', padding='same')(Conv13)
Maxp5 = MaxPooling2D(pool_size=(2,2))(Conv14)

flat = Flatten()(Maxp5)

FC1 = Dense(4096, activation='relu')(flat)
DO1 = Dropout(0.5)(FC1)
FC2 = Dense(4096, activation='relu')(DO1)
FC3 = Dense(38, activation='relu')(FC2)


model = Model(inputs=input1, outputs=FC3)

<p>Veamos algunas características del modelo:<p>

<h3>Resumen:</h3>


<img src="ResumenVGG19.png" alt="Drawing" style="width: 750px;"/>

<h3>Grafo:</h3>

<img src="modelVGG19.png" alt="Drawing" style="width: 450px;"/>





<p>Definamos ahora el optimizador, condiciones de parada (stopper) y métricas de corrida:</p>

In [ ]:
optimizer=keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#Callbacks: csv_logger: Registro de la corrida. stopper: Condiciones de parada antes de las 500 iteraciones.
csv_logger = keras.callbacks.CSVLogger('drive/training_Modelo1_VGG19.log')
stopper = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-08, patience=15, verbose=0, mode='auto')
model.compile(loss='mean_squared_error',optimizer=optimizer, metrics=['accuracy'])

<p>Ejecutemos el modelo:</p>

In [ ]:
model.fit(X_train, Y_train, batch_size=50, nb_epoch=500, verbose=1,callbacks=[csv_logger,stopper])
#Nota: El batch size es menor debido a que el modelo es demasiado pesado para manejar 128 imágenes a la vez
#con los recursos con los que se cuenta.

<h2>Evaluación del Test Set y visualización de resultados</h2>

<p>Después de finalizada la corrida, procedemos a evaluar el modelo con el test set:</p>

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Loss del Test Set: "+str(score[0]))

<p>Loss del Test Set: 0.0021535399928689003 </p>

<p>El modelo debe ser corrido varias veces para obtener un promedio de la pérdida real, aquí solo se muestra una vez.</p>

In [ ]:
#Función para visualizar una imágen con los labels reales (rojo) vs los predichos (azules)
def probador(start=1000):
    vector = model.predict(np.array(Diccionario['Img'][start:start+1]).reshape(1,128,128,1).astype('float32'),batch_size=1,verbose=0)
    plt.imshow(np.reshape(Diccionario['Img'][start:start+1][0],[128,128]),cmap='gray')
    Y_Predicted = np.array(vector,dtype='float32').reshape([19,2])
    return plt.scatter(x=Y_Predicted[:,0]*128,y=Y_Predicted[:,1]*128, c='b', s=4),plt.scatter(x=Diccionario['Lbl'][start:start+1][0].reshape([19,2])[:,0]*128,y=Diccionario['Lbl'][start:start+1][0].reshape([19,2])[:,1]*128, c='r', s=4)

<p>Algunas imágenes de resultado, recordar que los puntos rojos son los labels "reales" y los azules los labels "predichos":</p>



<img src="ImagenesVGG19/1014.png" style="width: 300px; float: left;"/>

<img src="ImagenesVGG19/1019.png" style="width: 300px; float: left;"/>

<img src="ImagenesVGG19/1036.png" style="width: 300px; float: left;"/>

<h4>Comentarios:</h4>

<p>El modelo presenta menor desempeño que el AlexNet, cabe destacar que existen numerosas razones por las cuales esto puede suceder, en mi opinión se debe a ajustes en los hiperparámetros de aprendizaje, especialmente el learning rate y el algoritmo de optimización. En pruebas separadas que he hecho con arquitecturas propias, ADAM tiene mejor desempeño que SGD con momentum.</p>